In [ ]:
import timm
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


(nog niet getest)

Doc gebruikt:

https://huggingface.co/docs/timm/models/mobilenet-v3

https://huggingface.co/docs/transformers/en/model_doc/gpt2

moblinenet


In [ ]:
model = timm.create_model('mobilenetv3_large_100', pretrained=True)
model.eval()

In [ ]:

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, filename)
img = Image.open(filename).convert('RGB')
tensor = transform(img).unsqueeze(0) # transform and add batch dimension

In [ ]:
with torch.no_grad():
    out = model(tensor)
probabilities = torch.nn.functional.softmax(out[0], dim=0)
print(probabilities.shape)
# prints: torch.Size([1000])

In [ ]:
# Get imagenet class mappings
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", "imagenet_classes.txt")
urllib.request.urlretrieve(url, filename) 
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Print top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())
# prints class names and probabilities like:
# [('Samoyed', 0.6425196528434753), ('Pomeranian', 0.04062102362513542), ('keeshond', 0.03186424449086189), ('white wolf', 0.01739676296710968), ('Eskimo dog', 0.011717947199940681)]

gpt2

In [ ]:

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

prompt = "GPT2 is a model developed by OpenAI."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]

Made by Copilot:

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained image model (ResNet)
image_model = models.resnet50(pretrained=True)
image_model.eval()

# Preprocess the image
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_image_features(image_path):
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        features = image_model(image)
    return features

# Load pre-trained language model (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
language_model = GPT2LMHeadModel.from_pretrained('gpt2')

def generate_text_from_image(image_path):
    # Get image features
    features = get_image_features(image_path)
    
    # Get top categories (assuming features are class probabilities)
    top5_prob, top5_catid = torch.topk(features, 5)
    
    # Convert categories to text
    categories_text = " ".join([str(catid.item()) for catid in top5_catid[0]])
    
    # Generate text using GPT-2
    inputs = tokenizer.encode(categories_text, return_tensors='pt')
    outputs = language_model.generate(inputs, max_length=50, num_return_sequences=1)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return text

# Example usage
image_path = 'imgs/duck.jpg'
generated_text = generate_text_from_image(image_path)
print(generated_text)